# **자기주도PJT - 심화과제**

**Req.7. 다음의 과정을 따라 advanced.ipynb에 VoxCeleb Dataset을 사용해 결과를 출력 해보세요.**
1. https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw 에서 동영상 파일 (예, 04.mp4) 하나를 다운로드 받아 구글 드라이브 first_motion_order_model 폴더에 복사해 놓습니다.
2. https://drive.google.com/drive/folders/1PyQJmkdCsAkOYwUyaj_l-l0as-iLDgeH 에서 다음의 Dataset 파일을 다운로드 받아 구글 드라이브 first_motion_order_model 폴더에 복사해 놓습니다.
  - vox-cpk.pth.tar
3. 휴대폰으로 셀카를 찍어 my_face.png 라는 파일명으로 구귿 드라이브 first_motion_order_model 폴더에 올려 놓습니다. (정사각형 형태로 촬영을 추천 합니다.)


In [1]:
# 구글 드라이브를 마운트 합니다.
from google.colab import drive
#drive.flush_and_unmount()  # 파일목록 갱신이 안될때는 이 부분 주석처리를 해제하고 실행하시기 바랍니다.
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 마운트된 드라이브의 first_order_motion_model 디렉토리의 파일목록을 불러옵니다.
!ls -l /content/drive/MyDrive/colab/first_motion_order_model

total 711904
-rw------- 1 root root    170054 Dec 28 07:49 10.mp4
-rw------- 1 root root     51606 Dec 28 08:00 my_face.png
-rw------- 1 root root 728766691 Dec 28 07:51 vox-cpk.pth.tar


In [3]:
# Aliaksandr Siarohin 의 github에서 First Order Model을 clone 합니다.
!git clone https://github.com/AliaksandrSiarohin/first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 337, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 337 (delta 15), reused 19 (delta 7), pack-reused 306
Receiving objects: 100% (337/337), 72.16 MiB | 22.39 MiB/s, done.
Resolving deltas: 100% (174/174), done.


In [14]:
# 디렉토리를 변경합니다.
%cd /content/first-order-model

/content/first-order-model


In [15]:
# 현재 디렉토리의 내용을 보여줍니다.
!ls -l

total 1276
-rw-r--r-- 1 root root    4061 Dec 28 08:02 animate.py
-rw-r--r-- 1 root root   12547 Dec 28 08:02 augmentation.py
drwxr-xr-x 2 root root    4096 Dec 28 08:02 config
-rw-r--r-- 1 root root    5876 Dec 28 08:02 crop-video.py
drwxr-xr-x 3 root root    4096 Dec 28 08:02 data
-rw-r--r-- 1 root root   22154 Dec 28 08:02 demo.ipynb
-rw-r--r-- 1 root root    7294 Dec 28 08:02 demo.py
-rw-r--r-- 1 root root     502 Dec 28 08:02 Dockerfile
-rw-r--r-- 1 root root    7027 Dec 28 08:02 frames_dataset.py
-rw-r--r-- 1 root root   18868 Dec 28 08:02 LICENSE.md
-rw-r--r-- 1 root root    8367 Dec 28 08:02 logger.py
drwxr-xr-x 2 root root    4096 Dec 28 08:02 modules
-rw-r--r-- 1 root root 1143882 Dec 28 08:02 old_demo.ipynb
-rw-r--r-- 1 root root    8727 Dec 28 08:02 README.md
-rw-r--r-- 1 root root    2772 Dec 28 08:02 reconstruction.py
-rw-r--r-- 1 root root     254 Dec 28 08:02 requirements.txt
-rw-r--r-- 1 root root    3299 Dec 28 08:02 run.py
drwxr-xr-x 2 root root    4096 Dec 28 08:02 

In [17]:
!pip install ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6084 sha256=27136f2c521ff4f331b829a3ea4f50523038feb0ae0d3e2d80d9f7a8638456b1
  Stored in directory: /root/.cache/pip/wheels/30/33/46/5ab7eca55b9490dddbf3441c68a29535996270ef1ce8b9b6d7
Successfully built ffmpeg


In [20]:
!pip install imageio-ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 26.9 MB 1.4 MB/s 


In [18]:
# 필요한 패키지들을 import 합니다.

import imageio

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.animation as animation

from skimage.transform import resize

from IPython.display import HTML

import warnings
warnings.filterwarnings('ignore')

from demo import load_checkpoints
from demo import make_animation
from skimage import img_as_ubyte

In [21]:
# 소스 이미지

source_image = imageio.imread('/content/drive/MyDrive/colab/first_motion_order_model/my_face.png')

# 소스 비디오

reader = imageio.get_reader('/content/drive/MyDrive/colab/first_motion_order_model/10.mp4')

# 이미지와 비디오크기를 256x256 픽셀로 변경 합니다
source_image = resize(source_image,(256,256))[..., :3]

# frames per second 1초당 표시할 프레임수를 가져옵니다
fps = reader.get_meta_data()['fps']



In [22]:
#driving_video를 초기화 하고 소스비디오의 각 프레임을 driving_video에 추가 합니다.

driving_video = []

try:
    
    for im in reader:
        
        driving_video.append(im)

except RuntimeError:
    
    pass

reader.close()

In [23]:
# driving_video를 256x256 으로 리사이즈 합니다.

driving_video = [resize(frame, (256,256))[...,:3] for frame in driving_video]

In [24]:
# 소스이미지와 기존 비디오를 합성해주는 display 함수

def display(source, driving, generated = None):
    
    fig = plt.figure(figsize = (8 + 4 *(generated is not None), 6))

    ims = []

    for i in range(len(driving)):
        
        cols = [source]

        cols.append(driving[i])

        if generated is not None:
            
            cols.append(generated[i])
        
        im = plt.imshow(np.concatenate(cols,axis=1), animated=True)

        plt.axis('off')

        ims.append([im])
    
    ani = animation.ArtistAnimation(fig, ims, interval = 50, repeat_delay=1000)

    plt.close()

    return ani

In [25]:
# display() 함수를 사용하여 HTML5 비디오 형태로 출력합니다.

HTML(display(source_image,driving_video).to_html5_video())

In [26]:
# VoxCeleb 모델의 체크포인트 불러오기 (700MB 파일이라 오래 걸립니다.)

generator, kp_detector = load_checkpoints(config_path = 'config/vox-256.yaml',
                                          checkpoint_path = '/content/drive/MyDrive/colab/first_motion_order_model/vox-cpk.pth.tar')

In [27]:
# make_animation() 함수를 사용하여 소스이미지, 소스비디오를 입력으로 predictions를 생성합니다.

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

# generated.mp4 파일로 생성된 결과를 저장

imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions],fps=fps)


# display 함수를 사용해 로 출력합니다.

HTML(display(source_image, driving_video, predictions).to_html5_video())



  0%|          | 0/444 [00:00<?, ?it/s]





---




**Req.8. 다음의 과정을 따라 advanced.ipynb에 Fashion-Videos Dataset을 사용해 결과를 출력 해보세요.**
1.	https://drive.google.com/drive/folders/1PyQJmkdCsAkOYwUyaj_l-l0as-iLDgeH 에서 다음의 Dataset 파일을 다운로드 받아 구글 드라이브 first_motion_order_model 폴더에 복사해 놓습니다.
  -	fashion.pth.tar
2.	https://vision.cs.ubc.ca/datasets/fashion/resources/fashion_dataset/fashion_train.txt 에서 4개의 mp4 파일을 다운로드 받습니다. 
3.	다운로드 받은 mp4파일을 전체화면으로 재생시킨 후 바로 Pause 버튼을 눌러 모델이 똑바로 앞을 보고 있는 장면에서 멈춘 후 컴퓨터의 화면 캡춰 프로그램을 사용해 동영상 화면을 캡춰해 m1,png, m2,png, mp3.png 등 3개의 정지화면을 얻은 후 구글드라이브에 복사해 놓습니다.
4.	나머지 1개의 동영상 혹은 해당 동영상과 비슷한 구도와 포즈의 전신 샷을 촬영하여 req8_source.mp4라는 이름으로 구글드라이브에 복사해 놓습니다.


In [28]:
from demo import load_checkpoints
from demo import make_animation
from skimage import img_as_ubyte

In [29]:
# 3개의 이미지와 1개의 동영상을 불러 옵니다.

src_img1 = imageio.imread('/content/drive/MyDrive/colab/first_motion_order_model/m1.png')
src_img2 = imageio.imread('/content/drive/MyDrive/colab/first_motion_order_model/m2.png')
src_img3 = imageio.imread('/content/drive/MyDrive/colab/first_motion_order_model/m3.png')

driving_video = imageio.mimread('/content/drive/MyDrive/colab/first_motion_order_model/req8_source.mp4.mp4', memtest=False)

In [30]:
# 이미지와 비디오를 256x256 사이즈로 변경 합니다.

src_img1 = resize(src_img1, (256,256))[..., :3]
src_img2 = resize(src_img2, (256,256))[..., :3]
src_img3 = resize(src_img3, (256,256))[..., :3]

driving_video = [resize(frame, (256,256))[..., :3] for frame in driving_video]

In [31]:
# load_checkpoints 함수를 사용해 이전에 학습해둔 fashion.pht.tar을 을 불러 옵니다.

generator, kp_detector = load_checkpoints(config_path = 'config/fashion-256.yaml',
                                          checkpoint_path = '/content/drive/MyDrive/colab/first_motion_order_model/fashion.pth.tar')

In [32]:
# 첫번째 모델의 사진과 Driving Video로 첫번째 모델의 움직이는 비디오를 만들고 재생합니다.

predictions1 = make_animation(src_img1, driving_video, generator, kp_detector, relative=True)
HTML(display(src_img1, driving_video, predictions1).to_html5_video())

  0%|          | 0/347 [00:00<?, ?it/s]

In [34]:
# 두번째 모델의 사진과 Driving Video로 두번째 모델의 움직이는 비디오를 만들고 재생합니다.

predictions2 = make_animation(src_img2, driving_video, generator, kp_detector, relative=True)
HTML(display(src_img2, driving_video, predictions2).to_html5_video())

  0%|          | 0/347 [00:00<?, ?it/s]

In [35]:
# 세번째 모델의 사진과 Driving Video로 세번째 모델의 움직이는 비디오를 만들고 재생합니다.

predictions3 = make_animation(src_img3, driving_video, generator, kp_detector, relative=True)
HTML(display(src_img3, driving_video, predictions3).to_html5_video())

  0%|          | 0/347 [00:00<?, ?it/s]